In [3]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# 使用Runnables来连接2个链

In [5]:
from operator import itemgetter  #获取可迭代对象中指定索引或键对应的元素

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("{person}来自于哪个城市?只返回城市名,其他不要返回")
prompt2 = ChatPromptTemplate.from_template("{city}属于哪个省? 用{language}来回答")

model = ChatOpenAI()
chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    # chain1的返回值直接作为city参数的值
    # itemgetter获取invoke中的language
        {"city": chain1, "language": itemgetter("language")}
        | prompt2
        | model
        | StrOutputParser()
)

In [6]:
chain1.invoke({"person": "马化腾"})
#chain2.invoke({"person": "马化腾", "language": "中文"})

'深圳'

In [8]:
chain2.invoke({"person": "马化腾", "language": "英文"})

'Shenzhen belongs to Guangdong province.'

# 连接多分支链

In [ ]:
     输入
/ \
    / \
    分支1   分支2
\   /
\ /
合并结果

In [9]:
from langchain_core.runnables import RunnablePassthrough

planner = (
        ChatPromptTemplate.from_template("生成一个关于{input}的论点")
        | ChatOpenAI() | StrOutputParser() | {"base_response": RunnablePassthrough()}
)
arguments_for = (
        ChatPromptTemplate.from_template("列出以下内容的优点或积极方面:{base_response}")
        | ChatOpenAI() | StrOutputParser()
)
arguments_against = (
        ChatPromptTemplate.from_template("列出以下内容的缺点或消极方面:{base_response}")
        | ChatOpenAI() | StrOutputParser()
)

final_responder = (
        ChatPromptTemplate.from_messages(
            [
                ("ai", "{original_response}"),
                ("human", "积极看法:\n{results_1}\n\n消极看法:\n{results_2}"),
                ("system", "根据上述评论生成最终的回复"),
            ]
        )
        | ChatOpenAI() | StrOutputParser()
)

chain = (
        planner
        | {
            "results_1": arguments_for,
            "results_2": arguments_against,
            "original_response": itemgetter("base_response"),
        }
        | final_responder
)

In [10]:
planner.invoke("生孩子")

{'base_response': '生孩子是人类社会中至关重要的事情，它不仅是延续种族的必然选择，更是家庭和社会发展的基石。通过生育子女，我们不仅可以传承血脉，延续家族的传统，更可以为社会注入新的活力和希望。同时，生育子女也是一种人类天性的体现，是对生命的尊重和热爱。因此，生孩子不仅是一种责任，更是一种荣耀和幸福。'}

In [11]:
chain.invoke({"input": "生孩子"})

'感谢评论者提供的意见和看法。的确，生孩子是一项需要认真考虑和权衡利弊的决定和责任。虽然生孩子可以带来很多幸福和满足感，但也需要付出巨大的精力、时间和金钱，并且存在着健康风险和社会影响等问题。因此，在做出生育决定之前，需要充分了解自己和家庭的情况，并认真考虑自己的生活和事业发展计划。同时，社会和政府也应该提供必要的支持和保障，为生育家庭创造更好的条件和环境。'

# 自定义输出解析器 - python代码生成工具

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

template = """根据用户需求帮助用户编写python代码. 
只需要返回markdown格式的python代码, 比如:
```python
def add(a, b):
    return a + b
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])
model = ChatOpenAI(model="gpt-4", temperature=0)


#自定义输出解析，只返回python代码
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]


#定义链
chain = prompt | model | StrOutputParser() | _sanitize_output

In [14]:
code = chain.invoke({"input": "从10里面循环找出所有质数"})
print(code)


def find_prime(n):
    prime_nums = []
    for num in range(2, n+1):
        if num > 1:
            for i in range(2, num):
                if (num % i) == 0:
                    break
            else:
                prime_nums.append(num)
    return prime_nums

print(find_prime(10))



In [19]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run
ans = chain.invoke({"input": "从10里面循环找出所有质数"})
print(ans)

[2, 3, 5, 7]



# 查询sql

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.get_table_info())


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Empl

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", temperature=0)


def get_schema(_):
    return db.get_table_info()


sql_response = (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        # 遇到SQLResult:就停止输出
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
)

In [24]:
sql_response.invoke({"question": "How many artists are there?"})

'SELECT COUNT(*) FROM Artist;'

也可以帮我们直接查询出来结果

In [27]:
from langchain.globals import set_debug


def run_query(query):
    print(query)
    return db.run(query)


template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
        RunnablePassthrough.assign(query=sql_response).assign(
            schema=get_schema,
            response=lambda x: db.run(x["query"]),
        )
        | prompt_response
        | model
)
set_debug(True)
full_chain.invoke({"question": "How many artists are there?"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many artists are there?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<query>] Entering Chain run with input:
{
  "question": "How many artists are there?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<query> > 3:chain:RunnableParallel<query>] Entering Chain run with input:
{
  "question": "How many artists are there?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<query> > 3:chain:RunnableParallel<query> > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many artists are there?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<query> > 3:chain:RunnableParallel<query> > 4:chain:RunnableSequence > 5:chain:RunnableAssign<schema>] Entering Chain run with input:
{
  "question": "How many artists are there?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<query> > 3:chain:R

AIMessage(content='There are 275 artists.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 2159, 'total_tokens': 2165}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f91978d1-a7cb-4771-9d26-69b47287cbd6-0')